# Create Elevation Index

Copernicus 30m DEM on AWS open data does not have a spatial index of the available tiles. We need to generate one, joining it to the spatial index provided. The polygon index can then be used to query for scenes matching an analysis aoi.